In [5]:
import time
import cv2
import psutil
import tracemalloc
from ultralytics import YOLO

# Start tracking memory
tracemalloc.start()

# Load the model and measure the time
start_time = time.time()
model = YOLO('./models/license_plate_detector.pt')  # Path to trained YOLOv8 model
model_load_time = time.time() - start_time

# Capture memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Model Load Time: {model_load_time:.4f} seconds")
print(f"Memory Usage During Model Load: Current = {current / 1024 / 1024:.2f} MB, Peak = {peak / 1024 / 1024:.2f} MB")

# Input image
input_image = cv2.imread('car_image.jpg')  # Path to the input image
assert input_image is not None, "Input image not found!"

# Inference and measure time
start_time = time.time()
results = model(input_image)
inference_time = time.time() - start_time

# Annotate detections
annotated_image = results[0].plot()

# Save the output image
output_path = './output_yolo.jpg'
cv2.imwrite(output_path, annotated_image)
print(f"Inference Time: {inference_time:.4f} seconds")
print(f"Output saved to: {output_path}")

Model Load Time: 0.1527 seconds
Memory Usage During Model Load: Current = 1.49 MB, Peak = 2.03 MB



0: 480x640 1 license_plate, 233.7ms
Speed: 9.0ms preprocess, 233.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


Inference Time: 2.5323 seconds
Output saved to: ./output_yolo.jpg


In [6]:
import time
import cv2
import psutil
import tracemalloc
import numpy as np
import onnxruntime as ort

# Start tracking memory
tracemalloc.start()

# Load ONNX model and measure the time
start_time = time.time()
onnx_model_path = './models/license_plate_detector.onnx'  # Path to ONNX model
ort_session = ort.InferenceSession(onnx_model_path)
model_load_time = time.time() - start_time

# Capture memory usage
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"ONNX Model Load Time: {model_load_time:.4f} seconds")
print(f"Memory Usage During Model Load: Current = {current / 1024 / 1024:.2f} MB, Peak = {peak / 1024 / 1024:.2f} MB")

# Input image
input_image = cv2.imread('car_image.jpg')  # Path to the input image
assert input_image is not None, "Input image not found!"

# Preprocess image for ONNX (resize and normalize)
input_shape = (640, 640)  # Assumes the model input size is 640x640
resized_image = cv2.resize(input_image, input_shape)
input_tensor = resized_image.transpose(2, 0, 1).astype('float32')  # Convert to CHW format
input_tensor = np.expand_dims(input_tensor, axis=0) / 255.0  # Add batch dimension and normalize

# Inference and measure time
start_time = time.time()
outputs = ort_session.run(None, {'images': input_tensor})
inference_time = time.time() - start_time

# Postprocess the outputs
detections = outputs[0][0]
for detection in detections:
    x, y, w, h, confidence, class_id = detection[:6]
    if confidence > 0.5:  # Confidence threshold
        x_min = int((x - w / 2) * input_image.shape[1] / input_shape[0])
        y_min = int((y - h / 2) * input_image.shape[0] / input_shape[1])
        x_max = int((x + w / 2) * input_image.shape[1] / input_shape[0])
        y_max = int((y + h / 2) * input_image.shape[0] / input_shape[1])
        
        # Draw bounding box
        cv2.rectangle(input_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        label = f"LP: {confidence:.2f}"
        cv2.putText(input_image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save the output image
output_path = './output_onnx.jpg'
cv2.imwrite(output_path, input_image)
print(f"Inference Time: {inference_time:.4f} seconds")
print(f"Output saved to: {output_path}")


ONNX Model Load Time: 0.3940 seconds
Memory Usage During Model Load: Current = 0.00 MB, Peak = 0.01 MB
Inference Time: 0.0843 seconds
Output saved to: ./output_onnx.jpg
